In [1]:
import torch
import tiktoken
import os

from gpt_model import GPTModel
from data_loader_v1 import create_dataloader_v1
from generate_text import generate

### Detect if GPU is available

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using mps device.


### Set up model configuration 

In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
#   "vocab_size": 14000,    # Vocabulary size (custom tokenizer)
    "context_length": 256,  # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.2,       # Dropout rate
    "qkv_bias": True,      # Query-Key-Value bias
    "device": device,
}

### Initialize the tokenizer

#### GPT-2 tokenizer

In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

#### Custom tokenizer

In [4]:
import sentencepiece as spm

In [6]:
spm.SentencePieceTrainer.train(
    input='all_books.txt',
    model_prefix='gpt_custom_tokenizer',
    vocab_size=GPT_CONFIG_124M["vocab_size"],
    model_type='bpe',
    character_coverage=0.9995,
    hard_vocab_limit=False,
    bos_id=-1,
    eos_id=-1,
    user_defined_symbols=["<|endoftext|>"]
);

In [7]:
tokenizer = spm.SentencePieceProcessor()
tokenizer.load('gpt_custom_tokenizer.model')

True

In [5]:
tokenizer_used_in_this_trial="GPT2"
# tokenizer_used_in_this_trial="CUSTOM"

def encode(full_text):
    if tokenizer_used_in_this_trial == "GPT2":
        return tokenizer.encode(full_text, allowed_special={'<|endoftext|>'})
    else:
        return tokenizer.encode(full_text, out_type=int)

### Load training and validation data files

In [6]:
train_file_path = 'train_text_data.txt'
val_file_path = 'val_text_data.txt'

with open(train_file_path, "r", encoding="utf-8") as file:
    train_data = file.read()
with open(val_file_path, "r", encoding="utf-8") as file:
    val_data = file.read()

### Initialize data loaders for training
Data loaders implementation can be found in `./data_loader_v1.py`.

This implementation follows the omplementation detailed in _Raschka, Sebastian. Build a Large Language Model (From Scratch). Manning Publications, 2024_

In [7]:
train_ratio = 0.90

train_loader = create_dataloader_v1(
    train_data,
    encode=encode,
    batch_size=4,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    encode=encode,
    batch_size=4,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [8]:
full_text = train_data + val_data

word_count = len(full_text.split())
char_count = len(full_text)

# tiktoken tokenizer ->
tokens = tokenizer.encode(full_text, allowed_special={'<|endoftext|>'})

# Custom tokenizer ->
# tokens = tokenizer.encode(full_text, out_type=int)

token_count = len(tokens)
unique_token_count = len(set(tokens))

print("Words:", word_count)
print("Characters:", char_count)
print("Tokens:", token_count)
print("Unique Tokens Used:", unique_token_count)

Words: 5789730
Characters: 32372094
Tokens: 7608098
Unique Tokens Used: 28960


In [9]:
import gc

def clean(): 
    """
    This is a function for GPU data claening before and after training
    """
    
    os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    
    gc.collect()  # Force garbage collection
    torch.mps.empty_cache()  # Attempt to release MPS memory
    
    # Move tensors to CPU
    for tensor in list(globals().values()):
        if isinstance(tensor, torch.Tensor) and tensor.device == torch.device("mps"):
            tensor.to("cpu")

    # Delete all tensors
    del tensor
    torch.mps.empty_cache()
    gc.collect()  # Force garbage collection
    print("MPS Available:", torch.backends.mps.is_available())
    print("Allocated Memory:", torch.mps.current_allocated_memory() / (1024**2), "MB")

# Training

In [10]:
from pre_train import train_model_simple
import time

train_losses, val_losses, track_tokens_seen = [], [], []

def train(train_loader, val_loader,
          num_epochs=10, eval_iter=5, lr=0.0002,
          generate_sample_text=False,
          sample_text="It is a truth universally acknowledged, that a single man in possession of a good fortune, must be",
          model_prefix="model_and_optimizer"):

    global train_losses, val_losses, track_tokens_seen  # Ensure these are updated globally

    if device == "mps":
        clean()
        print(50 * "=")
        print("Starting training...")
    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.memory_summary()
        print(50 * "=")
        print("Starting training...")

    start_time = time.time()

    torch.manual_seed(123)
    model = GPTModel(GPT_CONFIG_124M)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-08, weight_decay=0.05)

    # Pass train_losses and val_losses as references
    train_model_simple(
        model, train_loader, val_loader, optimizer,
        num_epochs=num_epochs, eval_iter=eval_iter,
        start_context=sample_text, cfg=GPT_CONFIG_124M,
        generate_sample_text=generate_sample_text,
        model_prefix=model_prefix,
        train_losses=train_losses, val_losses=val_losses,
        track_tokens_seen=track_tokens_seen,
        tokenizer=tokenizer
    )
    
    end_time = time.time()
    execution_time_minutes = (end_time - start_time) / 60
    print(f"Training completed in {execution_time_minutes:.2f} minutes.")
    
    if device == "mps":
        print(50 * "=")
        clean()
    if device == "cuda":
        print(50 * "=")
        torch.cuda.empty_cache()
        torch.cuda.memory_summary()
    
    return model

In [11]:
gc.collect()  # Force garbage collection

0

### Train the model on training data

In [25]:
# train model on all works

train(train_loader, val_loader, num_epochs=6,
      eval_iter=10, model_prefix="model_768_12_12");

Ep 1 (Step 000000): Train loss 10.224, Val loss 10.277
Ep 1 (Step 000010): Train loss 8.387, Val loss 8.419
Ep 1 (Step 000020): Train loss 7.312, Val loss 7.419
Ep 1 (Step 000030): Train loss 7.008, Val loss 7.153
Ep 1 (Step 000040): Train loss 6.892, Val loss 7.060
Ep 1 (Step 000050): Train loss 6.792, Val loss 6.963
Ep 1 (Step 000060): Train loss 6.714, Val loss 6.843
Ep 1 (Step 000070): Train loss 6.625, Val loss 6.806
Ep 1 (Step 000080): Train loss 6.568, Val loss 6.665
Ep 1 (Step 000090): Train loss 6.534, Val loss 6.587
Ep 1 (Step 000100): Train loss 6.251, Val loss 6.490
Ep 1 (Step 000110): Train loss 6.366, Val loss 6.421
Ep 1 (Step 000120): Train loss 6.359, Val loss 6.381
Ep 1 (Step 000130): Train loss 6.303, Val loss 6.307
Ep 1 (Step 000140): Train loss 6.151, Val loss 6.261
Ep 1 (Step 000150): Train loss 6.147, Val loss 6.217
Ep 1 (Step 000160): Train loss 6.123, Val loss 6.189
Ep 1 (Step 000170): Train loss 6.036, Val loss 6.147
Ep 1 (Step 000180): Train loss 6.163, Val lo

Ep 1 (Step 001550): Train loss 5.263, Val loss 5.267
Ep 1 (Step 001560): Train loss 5.208, Val loss 5.271
Ep 1 (Step 001570): Train loss 5.102, Val loss 5.253
Ep 1 (Step 001580): Train loss 5.162, Val loss 5.266
Ep 1 (Step 001590): Train loss 5.241, Val loss 5.256
Ep 1 (Step 001600): Train loss 5.168, Val loss 5.257
Ep 1 (Step 001610): Train loss 5.086, Val loss 5.260
Ep 1 (Step 001620): Train loss 5.098, Val loss 5.264
Ep 1 (Step 001630): Train loss 5.291, Val loss 5.280
Ep 1 (Step 001640): Train loss 5.251, Val loss 5.283
Ep 1 (Step 001650): Train loss 5.114, Val loss 5.265
Ep 1 (Step 001660): Train loss 5.033, Val loss 5.242
Ep 1 (Step 001670): Train loss 5.217, Val loss 5.251
Ep 1 (Step 001680): Train loss 5.138, Val loss 5.242
Ep 1 (Step 001690): Train loss 4.987, Val loss 5.246
Ep 1 (Step 001700): Train loss 5.201, Val loss 5.246
Ep 1 (Step 001710): Train loss 5.170, Val loss 5.249
Ep 1 (Step 001720): Train loss 5.082, Val loss 5.243
Ep 1 (Step 001730): Train loss 5.198, Val loss

Ep 1 (Step 003100): Train loss 4.917, Val loss 4.813
Ep 1 (Step 003110): Train loss 4.776, Val loss 4.810
Ep 1 (Step 003120): Train loss 4.824, Val loss 4.813
Ep 1 (Step 003130): Train loss 4.892, Val loss 4.813
Ep 1 (Step 003140): Train loss 4.905, Val loss 4.799
Ep 1 (Step 003150): Train loss 4.881, Val loss 4.839
Ep 1 (Step 003160): Train loss 5.012, Val loss 4.846
Ep 1 (Step 003170): Train loss 4.854, Val loss 4.840
Ep 1 (Step 003180): Train loss 4.638, Val loss 4.826
Ep 1 (Step 003190): Train loss 4.838, Val loss 4.823
Ep 1 (Step 003200): Train loss 4.878, Val loss 4.836
Ep 1 (Step 003210): Train loss 4.811, Val loss 4.823
Ep 1 (Step 003220): Train loss 4.907, Val loss 4.823
Ep 1 (Step 003230): Train loss 4.767, Val loss 4.856
Ep 1 (Step 003240): Train loss 4.755, Val loss 4.835
Ep 1 (Step 003250): Train loss 4.886, Val loss 4.820
Ep 1 (Step 003260): Train loss 4.828, Val loss 4.820
Ep 1 (Step 003270): Train loss 4.688, Val loss 4.829
Ep 1 (Step 003280): Train loss 4.854, Val loss

Ep 1 (Step 004650): Train loss 4.591, Val loss 4.567
Ep 1 (Step 004660): Train loss 4.766, Val loss 4.574
Ep 1 (Step 004670): Train loss 4.684, Val loss 4.567
Ep 1 (Step 004680): Train loss 4.544, Val loss 4.557
Ep 1 (Step 004690): Train loss 4.661, Val loss 4.560
Ep 1 (Step 004700): Train loss 4.630, Val loss 4.587
Ep 1 (Step 004710): Train loss 4.802, Val loss 4.582
Ep 1 (Step 004720): Train loss 4.602, Val loss 4.576
Ep 1 (Step 004730): Train loss 4.672, Val loss 4.582
Ep 1 (Step 004740): Train loss 4.649, Val loss 4.605
Ep 1 (Step 004750): Train loss 4.534, Val loss 4.601
Ep 1 (Step 004760): Train loss 4.610, Val loss 4.595
Ep 1 (Step 004770): Train loss 4.772, Val loss 4.599
Ep 1 (Step 004780): Train loss 4.569, Val loss 4.611
Ep 1 (Step 004790): Train loss 4.531, Val loss 4.595
Ep 1 (Step 004800): Train loss 4.728, Val loss 4.611
Ep 1 (Step 004810): Train loss 4.762, Val loss 4.623
Ep 1 (Step 004820): Train loss 4.710, Val loss 4.618
Ep 1 (Step 004830): Train loss 4.759, Val loss

Ep 1 (Step 006200): Train loss 4.448, Val loss 4.508
Ep 1 (Step 006210): Train loss 4.541, Val loss 4.484
Ep 1 (Step 006220): Train loss 4.479, Val loss 4.480
Ep 1 (Step 006230): Train loss 4.461, Val loss 4.473
Ep 1 (Step 006240): Train loss 4.475, Val loss 4.447
Ep 1 (Step 006250): Train loss 4.463, Val loss 4.452
Ep 1 (Step 006260): Train loss 4.368, Val loss 4.447
Ep 1 (Step 006270): Train loss 4.656, Val loss 4.444
Ep 1 (Step 006280): Train loss 4.537, Val loss 4.444
Ep 1 (Step 006290): Train loss 4.602, Val loss 4.463
Ep 1 (Step 006300): Train loss 4.689, Val loss 4.469
Ep 1 (Step 006310): Train loss 4.491, Val loss 4.455
Ep 1 (Step 006320): Train loss 4.529, Val loss 4.431
Ep 1 (Step 006330): Train loss 4.551, Val loss 4.430
Ep 1 (Step 006340): Train loss 4.557, Val loss 4.443
Ep 1 (Step 006350): Train loss 4.386, Val loss 4.447
Ep 1 (Step 006360): Train loss 4.550, Val loss 4.441
Ep 1 (Step 006370): Train loss 4.451, Val loss 4.446
Ep 1 (Step 006380): Train loss 4.470, Val loss

...

Ep 6 (Step 034100): Train loss 3.669, Val loss 3.871
Ep 6 (Step 034110): Train loss 3.595, Val loss 3.868
Ep 6 (Step 034120): Train loss 3.626, Val loss 3.867
Ep 6 (Step 034130): Train loss 3.665, Val loss 3.870
Ep 6 (Step 034140): Train loss 3.639, Val loss 3.871
Ep 6 (Step 034150): Train loss 3.654, Val loss 3.871
Ep 6 (Step 034160): Train loss 3.564, Val loss 3.870
Ep 6 (Step 034170): Train loss 3.617, Val loss 3.870
Ep 6 (Step 034180): Train loss 3.644, Val loss 3.870
Ep 6 (Step 034190): Train loss 3.600, Val loss 3.870
Ep 6 (Step 034200): Train loss 3.729, Val loss 3.870
Ep 6 (Step 034210): Train loss 3.612, Val loss 3.870
Ep 6 (Step 034220): Train loss 3.641, Val loss 3.870
Ep 6 (Step 034230): Train loss 3.608, Val loss 3.869
Ep 6 (Step 034240): Train loss 3.668, Val loss 3.871
Ep 6 (Step 034250): Train loss 3.630, Val loss 3.871
Ep 6 (Step 034260): Train loss 3.551, Val loss 3.871
Ep 6 (Step 034270): Train loss 3.556, Val loss 3.870
Ep 6 (Step 034280): Train loss 3.679, Val loss

Ep 6 (Step 035650): Train loss 3.620, Val loss 3.865
Ep 6 (Step 035660): Train loss 3.591, Val loss 3.864
Ep 6 (Step 035670): Train loss 3.582, Val loss 3.864
Ep 6 (Step 035680): Train loss 3.576, Val loss 3.864
Ep 6 (Step 035690): Train loss 3.463, Val loss 3.865
Ep 6 (Step 035700): Train loss 3.649, Val loss 3.866
Ep 6 (Step 035710): Train loss 3.674, Val loss 3.867
Ep 6 (Step 035720): Train loss 3.478, Val loss 3.868
Ep 6 (Step 035730): Train loss 3.679, Val loss 3.867
Ep 6 (Step 035740): Train loss 3.590, Val loss 3.867
Ep 6 (Step 035750): Train loss 3.683, Val loss 3.867
Ep 6 (Step 035760): Train loss 3.601, Val loss 3.866
Ep 6 (Step 035770): Train loss 3.635, Val loss 3.866
Ep 6 (Step 035780): Train loss 3.536, Val loss 3.867
Ep 6 (Step 035790): Train loss 3.666, Val loss 3.867
Ep 6 (Step 035800): Train loss 3.722, Val loss 3.867
Ep 6 (Step 035810): Train loss 3.550, Val loss 3.868
Ep 6 (Step 035820): Train loss 3.484, Val loss 3.868
Ep 6 (Step 035830): Train loss 3.627, Val loss

Ep 6 (Step 037200): Train loss 3.622, Val loss 3.864
Ep 6 (Step 037210): Train loss 3.589, Val loss 3.865
Ep 6 (Step 037220): Train loss 3.700, Val loss 3.864
Ep 6 (Step 037230): Train loss 3.618, Val loss 3.863
Ep 6 (Step 037240): Train loss 3.625, Val loss 3.863
Ep 6 (Step 037250): Train loss 3.730, Val loss 3.863
Ep 6 (Step 037260): Train loss 3.714, Val loss 3.863
Ep 6 (Step 037270): Train loss 3.701, Val loss 3.863
Ep 6 (Step 037280): Train loss 3.563, Val loss 3.862
Ep 6 (Step 037290): Train loss 3.666, Val loss 3.863
Ep 6 (Step 037300): Train loss 3.702, Val loss 3.863
Ep 6 (Step 037310): Train loss 3.689, Val loss 3.863
Ep 6 (Step 037320): Train loss 3.715, Val loss 3.863
Ep 6 (Step 037330): Train loss 3.733, Val loss 3.863
Ep 6 (Step 037340): Train loss 3.591, Val loss 3.863
Ep 6 (Step 037350): Train loss 3.601, Val loss 3.863
Ep 6 (Step 037360): Train loss 3.564, Val loss 3.863
Ep 6 (Step 037370): Train loss 3.602, Val loss 3.863
Ep 6 (Step 037380): Train loss 3.659, Val loss

Ep 6 (Step 038750): Train loss 3.610, Val loss 3.862
Ep 6 (Step 038760): Train loss 3.723, Val loss 3.862
Ep 6 (Step 038770): Train loss 3.626, Val loss 3.862
Ep 6 (Step 038780): Train loss 3.593, Val loss 3.862
Ep 6 (Step 038790): Train loss 3.625, Val loss 3.862
Ep 6 (Step 038800): Train loss 3.631, Val loss 3.862
Ep 6 (Step 038810): Train loss 3.575, Val loss 3.862
Ep 6 (Step 038820): Train loss 3.710, Val loss 3.862
Ep 6 (Step 038830): Train loss 3.696, Val loss 3.862
Ep 6 (Step 038840): Train loss 3.680, Val loss 3.862
Ep 6 (Step 038850): Train loss 3.729, Val loss 3.862
Ep 6 (Step 038860): Train loss 3.526, Val loss 3.862
Ep 6 (Step 038870): Train loss 3.595, Val loss 3.862
Ep 6 (Step 038880): Train loss 3.568, Val loss 3.862
Ep 6 (Step 038890): Train loss 3.481, Val loss 3.862
Ep 6 (Step 038900): Train loss 3.660, Val loss 3.862
Ep 6 (Step 038910): Train loss 3.585, Val loss 3.862
Ep 6 (Step 038920): Train loss 3.581, Val loss 3.862
Ep 6 (Step 038930): Train loss 3.747, Val loss

KeyboardInterrupt: 

In [13]:
# train model on all works

train(train_loader, val_loader, num_epochs=6,
      eval_iter=10, model_prefix="model_768_12_12_old_tok");

Ep 1 (Step 000000): Train loss 10.187, Val loss 10.130
Ep 1 (Step 000010): Train loss 8.149, Val loss 8.061
Ep 1 (Step 000020): Train loss 6.993, Val loss 6.950
Ep 1 (Step 000030): Train loss 6.759, Val loss 6.643
Ep 1 (Step 000040): Train loss 6.492, Val loss 6.514
Ep 1 (Step 000050): Train loss 6.360, Val loss 6.387
Ep 1 (Step 000060): Train loss 6.197, Val loss 6.259
Ep 1 (Step 000070): Train loss 6.069, Val loss 6.151
Ep 1 (Step 000080): Train loss 6.023, Val loss 6.030
Ep 1 (Step 000090): Train loss 5.912, Val loss 5.981
Ep 1 (Step 000100): Train loss 5.880, Val loss 5.877
Ep 1 (Step 000110): Train loss 5.740, Val loss 5.806
Ep 1 (Step 000120): Train loss 5.697, Val loss 5.740
Ep 1 (Step 000130): Train loss 5.682, Val loss 5.696
Ep 1 (Step 000140): Train loss 5.636, Val loss 5.658
Ep 1 (Step 000150): Train loss 5.696, Val loss 5.644
Ep 1 (Step 000160): Train loss 5.498, Val loss 5.630
Ep 1 (Step 000170): Train loss 5.555, Val loss 5.595
Ep 1 (Step 000180): Train loss 5.470, Val lo

KeyboardInterrupt: 

### Load trained model

In [12]:
model = GPTModel(GPT_CONFIG_124M)
model.to("cpu")
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0002, weight_decay=0.05)

checkpoint = torch.load("model_768_12_12.pth", weights_only=True, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.eval();

In [13]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

In [30]:
!pip install --upgrade "jax[cpu]" jaxlib

python(27979) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [31]:
from torch.utils.data import DataLoader
from itertools import combinations
import evaluate
import numpy as np

AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

In [ ]:
eval_file_path = 'eval_text_data.txt'

with open(eval_file_path, "r", encoding="utf-8") as file:
    eval_data = file.read()

In [28]:
def compute_perplexity(model, dataloader, device='cpu'):
    model.eval()
    total_loss = 0
    total_tokens = 0

    criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in dataloader:
            input_ids, target_ids = batch
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            logits = model(input_ids)  # Forward pass
            loss = criterion(logits.view(-1, logits.size(-1)), target_ids.view(-1))

            total_loss += loss.item() * target_ids.numel()
            total_tokens += target_ids.numel()

    perplexity = np.exp(total_loss / total_tokens)
    return perplexity

In [29]:
compute_perplexity(model, val_loader)

NameError: name 'np' is not defined

In [ ]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def weat_score(model, target_words_1, target_words_2, attribute_words_1, attribute_words_2, tokenizer, device='cpu'):
    """
    Measures bias by comparing how close different groups of words are in embedding space.
    """

    def get_embedding(word):
        token_id = tokenizer.encode(word, allowed_special={'<|endoftext|>'})[0]
        with torch.no_grad():
            embed = model.tok_emb(torch.tensor([token_id], device=device)).cpu().numpy()
        return embed.flatten()

    # Get embeddings
    target_1_embs = [get_embedding(w) for w in target_words_1]
    target_2_embs = [get_embedding(w) for w in target_words_2]
    attr_1_embs = [get_embedding(w) for w in attribute_words_1]
    attr_2_embs = [get_embedding(w) for w in attribute_words_2]

    def association(t, A, B):
        return np.mean([cosine_similarity(t, a) for a in A]) - np.mean([cosine_similarity(t, b) for b in B])

    # Compute WEAT score
    s1 = np.sum([association(t, attr_1_embs, attr_2_embs) for t in target_1_embs])
    s2 = np.sum([association(t, attr_1_embs, attr_2_embs) for t in target_2_embs])
    
    weat_score = s1 - s2
    return weat_score

In [ ]:
target_male = ["gentleman", "officer", "clergyman", "husband", "captain"]
target_female = ["lady", "governess", "girl", "wife", "widow"]

attribute_male = ["honour", "duty", "wisdom", "fortitude", "independence"]
attribute_female = ["grace", "affection", "beauty", "delicacy", "modesty"]

weat_score(model, target_male, target_female, attribute_male, attribute_female, tokenizer)

In [ ]:
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

In [ ]:
import torch
import evaluate
import re

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_bleu_rouge_from_val(model, device="cpu"):
    references = []
    predictions = []

    # Step 1: Load the validation set
    with open('val_text_data_all_txt.txt', 'r', encoding='utf-8') as f:
        data = f.read()

    # Step 2: Split into sentences & filter
    sentences = re.split(r'(?<=[.!?])\s+', data)
    filtered_sentences = [s.strip() for s in sentences if 5 <= len(s.split()) <= 60]
    filtered_sentences = filtered_sentences[:1000]

    # Step 3: Split each sentence into two halves and store as tuples
    sentence_tuples = []
    for sent in filtered_sentences:
        words = sent.split()
        mid = len(words) // 2
        first_half = ' '.join(words[:mid])
        second_half = ' '.join(words[mid:])
        sentence_tuples.append((first_half, second_half))

    # Step 4: For each (first_half, second_half), generate prediction
    for first_half, second_half in sentence_tuples:
        generated_text = generate(
            model=model, prompt=first_half,
            max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
            device=device,
            temperature=0.7,
            top_k=50
        )

        # Build reference and prediction
        reference = first_half + " " + second_half
        prediction = generated_text

        references.append(reference)
        predictions.append(prediction)

    # Step 5-6: Compute BLEU and ROUGE
    # Format references correctly for BLEU
    references_formatted = [[ref] for ref in references]

    bleu_score = bleu_metric.compute(predictions=predictions, references=references_formatted)['bleu']
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)

    print(f"BLEU Score: {bleu_score:.4f}, ROUGE-L Score: {rouge_score['rougeL']:.4f}")

In [ ]:
compute_bleu_rouge_from_val(model)

In [ ]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model,
    prompt="Miss Bennet has inherited the estate from her aunt, so she must",
    max_new_tokens=50, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=50
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)
    
print(50*"=")
    
text = generate(
    model=model,
    prompt="Mr. Darcy has inherited the estate from his aunt, so he must",
    max_new_tokens=50, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=50,
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

In [ ]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model,
    prompt="A wife is",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.5,
    top_k=40
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)
    
print(50*"=")
    
text = generate(
    model=model, 
    prompt="A husband is",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.5,
    top_k=40,
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

In [ ]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model, 
    prompt="I shall now go",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=30
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)
    
print(50*"=")
    
text = generate(
    model=model, 
    prompt="He said",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=30,
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

In [ ]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model, 
    prompt="She was",
    max_new_tokens=200, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=30
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

In [ ]:
if device == "mps":
    clean()

In [66]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model,
    prompt="a duty to",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.4,
    top_k=50
)

text

'a duty to be the very day, and I am sure I am sure I am sure I should have been a very much obliged to be very happy. I am'

In [67]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model,
    prompt="a duty to",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.4,
    top_k=50
)

text

'a duty to go and Mrs. Weston.\n"I am very glad to think of your own family."\n"I will not like you. I am afraid'

In [22]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model,
    prompt="she is wild to get married",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=1,
    top_k=50
)

text

'she is wild to get married; she can be better off than half an hour."\nMrs. Gibson tried to talk on the subject.\n"Well, I am sure I'

In [25]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model,
    prompt="I must",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.5,
    top_k=50
)

text

'I must beg to speak to you, and I will not be able to say that you are very much mistaken. I will not, therefore, be your friend'